# Title : Data Wrangling on Real Estate Market

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
# from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("datasets/Bengaluru_House_Data.csv") 

In [ ]:
df.head()

In [ ]:
df.shape 

In [ ]:
df.columns

In [ ]:
df['area_type']

In [ ]:
df['area_type'].unique()

In [ ]:
df['area_type'].value_counts()

In [ ]:
df1 = df.drop(['area_type','society','balcony','availability'],axis='columns') 

In [ ]:
df1.shape

In [ ]:
df1.isnull().sum()

In [ ]:
df1.shape 

In [ ]:
df2 = df1.dropna() 
df2.isnull().sum()

In [ ]:
df2.shape 

In [ ]:
df2['size'].unique()

In [ ]:
df2['bhk'] = df2['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
df2.head()

In [ ]:
df2.bhk.unique()

In [ ]:
df2[df2.bhk>20]

In [ ]:
df2.total_sqft.unique()

In [ ]:
def is_float(x):
    try:
        float(x) 
        return True
    except(ValueError, TypeError):
        return False 

In [ ]:
df2[~df2['total_sqft'].apply(is_float)].head(10)

In [ ]:
def convert_sqft_to_num(x): 
    tokens = x.split('-')
    if len(tokens) == 2:
        try:
            return (float(tokens[0])+float(tokens[1]))/2
        except ValueError:
            return None
    try:
        return float(x) 
    except ValueError:
        return None 
    
result = convert_sqft_to_num('2100 - 2850')
print(result)

In [ ]:
convert_sqft_to_num('34.46Sq. Meter') 
df3 = df2.copy()
df3.total_sqft = df3.total_sqft.apply(convert_sqft_to_num) 
df3

In [ ]:
df3 = df3[df3.total_sqft.notnull()] 
df3

In [ ]:
df3.loc[30]

In [ ]:
df4 = df3.copy()
df4['price_per_sqft'] = df4['price']*100000/df4['total_sqft'] 
df4.head()

In [ ]:
df4_stats = df4['price_per_sqft'].describe() 
df4_stats

In [ ]:
df4.to_csv("./datasets/bhp.csv",index=False)

In [ ]:
df4.location = df4.location.apply(lambda x: x.strip()) 
location_stats = df4['location'].value_counts(ascending=False) 
location_stats

In [ ]:
len(location_stats[location_stats>10])

In [ ]:
len(location_stats) 

In [ ]:
len(location_stats[location_stats<=10]) 

In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10] 
location_stats_less_than_10

In [ ]:
len(df4.location.unique())

In [ ]:
df4.location = df4.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x) 
len(df4.location.unique())

In [ ]:
df4.head(10)

In [ ]:
df4[df4.total_sqft/df4.bhk<300].head()

In [ ]:
df4.shape 

In [ ]:
df5 = df4[~(df4.total_sqft/df4.bhk<300)] 
df5.shape

In [ ]:
df5.columns

In [ ]:
def plot_boxplot(df, column):
    plt.boxplot(df[column])
    plt.title(f"{column}")
    plt.show()

In [ ]:
def remove_outliers(df, column):
    Q1 = np.percentile(df[column], 25.)
    Q3 = np.percentile(df[column], 75.)
    IQR = Q3 - Q1
    ll = Q1 - (1.5 * IQR)
    ul = Q3 + (1.5 * IQR)
    bad_indices = df[(df[column] < ll) | (df[column] > ul)].index.tolist()
    df_out= df.drop(bad_indices)
    print("Before")
    plot_boxplot(df, column)
    print("After")
    plot_boxplot(df_out, column)
    return df

In [ ]:
columns_to_check = ['total_sqft', 'bath', 'price', 'bhk', 'price_per_sqft']
for column in columns_to_check:
    remove_outliers(df5, column)
